In [1]:
import utils
inp = utils.get_input(2019, 12)[:-1]
# inp = """<x=-1, y=0, z=2>
# <x=2, y=-10, z=-7>
# <x=4, y=-8, z=8>
# <x=3, y=5, z=-1>"""
# inp = """<x=-1, y=0, z=2>
# <x=2, y=-10, z=-7>
# <x=4, y=-8, z=8>
# <x=3, y=5, z=-1>"""
print(inp)

Found cached input for 2019 day 12
<x=16, y=-8, z=13>
<x=4, y=10, z=10>
<x=17, y=-5, z=6>
<x=13, y=-3, z=0>


In [2]:
class memoize(dict):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self[args]

    def __missing__(self, key):
        result = self[key] = self.func(*key)
        return result

In [32]:
import re
from copy import deepcopy
from pprint import pprint
from itertools import repeat, permutations, product
from collections import namedtuple

coord = namedtuple('coord', ['x', 'y', 'z'])

moons = list(map(lambda x: coord(*map(int, x.split(','))), re.sub(r'[<>=xyz ]', '', inp).split('\n')))
moons = dict(enumerate(zip(moons, repeat(coord(0, 0, 0)))))
antis = deepcopy(moons)
# print(moons)

pairs = set()
for p in product([0, 1, 2, 3], repeat=2):
    if (p not in pairs) and ((p[1], p[0]) not in pairs):
        pairs.add(p)

@memoize
def adj(c, x=0, y=0, z=0):
    return coord(c.x + x, c.y + y, c.z + z)

def update_gravity(a, b, mult=1):
    if a == b:
        return
    for dim in ['x', 'y', 'z']:
        if getattr(moons[a][0], dim) > getattr(moons[b][0], dim):
            moons[a] = (moons[a][0], adj(moons[a][1], **{dim: -1 * mult}))
            moons[b] = (moons[b][0], adj(moons[b][1], **{dim: 1 * mult}))
        elif getattr(moons[a][0], dim) < getattr(moons[b][0], dim):
            moons[a] = (moons[a][0], adj(moons[a][1], **{dim: 1 * mult}))
            moons[b] = (moons[b][0], adj(moons[b][1], **{dim: -1 * mult}))

def update_antigravity(a, b):
    if a == b:
        return
    for dim in ['x', 'y', 'z']:
        if getattr(antis[a][0], dim) > getattr(antis[b][0], dim):
            antis[a] = (antis[a][0], adj(antis[a][1], **{dim: -1}))
            antis[b] = (antis[b][0], adj(antis[b][1], **{dim: 1}))
        elif getattr(antis[a][0], dim) < getattr(antis[b][0], dim):
            antis[a] = (antis[a][0], adj(antis[a][1], **{dim: 1}))
            antis[b] = (antis[b][0], adj(antis[b][1], **{dim: -1}))

def update_velocity(m):
    moons[m] = (adj(moons[m][0], *moons[m][1]), moons[m][1])

def update_antivelocity(m):
    antis[m] = (adj(antis[m][0], *antis[m][1]), antis[m][1])

def energy(m):
    def _pot():
        return list(map(lambda p: sum(map(abs, p[0])), m.values()))
    def _kin():
        return list(map(lambda p: sum(map(abs, p[1])), m.values()))
    return sum([p * k for p, k in zip(_pot(), _kin())])
    
orig = deepcopy(moons)
i = 0
mem = {i: energy(moons)}
prev = energy(moons)
# while True:
#     for p in pairs:
#         update_gravity(*p)
# #         update_antigravity(*p)
#     for m in moons:
#         update_velocity(m)
# #         update_antivelocity(m)

#     i += 1
#     if energy(moons) in mem.values():
#         prev = [k for k, v in mem.items() if v == energy(moons)][0]
#         print(energy(moons), i, prev, (i - prev))
#         if moons == orig:
#             break
#     else:
#         mem[i] = energy(moons)

#     if i % 10000 == 0:
#         print(i)
#         print(len(mem))
#     print(i, prev - energy(moons))
# print(energy(moons))
# pprint(moons)
for i in range(1000):
    for p in pairs:
        update_gravity(*p)
    for m in moons:
        update_velocity(m)
    pprint(moons)
# print(energy(moons))
# pprint(moons)

{0: (coord(x=2, y=-1, z=1), coord(x=3, y=-1, z=-1)),
 1: (coord(x=3, y=-7, z=-4), coord(x=1, y=3, z=3)),
 2: (coord(x=1, y=-7, z=5), coord(x=-3, y=1, z=-3)),
 3: (coord(x=2, y=2, z=0), coord(x=-1, y=-3, z=1))}
{0: (coord(x=5, y=-3, z=-1), coord(x=3, y=-2, z=-2)),
 1: (coord(x=1, y=-2, z=2), coord(x=-2, y=5, z=6)),
 2: (coord(x=1, y=-4, z=-1), coord(x=0, y=3, z=-6)),
 3: (coord(x=1, y=-4, z=2), coord(x=-1, y=-6, z=2))}
{0: (coord(x=5, y=-6, z=-1), coord(x=0, y=-3, z=0)),
 1: (coord(x=0, y=0, z=6), coord(x=-1, y=2, z=4)),
 2: (coord(x=2, y=1, z=-5), coord(x=1, y=5, z=-4)),
 3: (coord(x=1, y=-8, z=2), coord(x=0, y=-4, z=0))}
{0: (coord(x=2, y=-8, z=0), coord(x=-3, y=-2, z=1)),
 1: (coord(x=2, y=1, z=7), coord(x=2, y=1, z=1)),
 2: (coord(x=2, y=3, z=-6), coord(x=0, y=2, z=-1)),
 3: (coord(x=2, y=-9, z=1), coord(x=1, y=-1, z=-1))}
{0: (coord(x=-1, y=-9, z=2), coord(x=-3, y=-1, z=2)),
 1: (coord(x=4, y=1, z=5), coord(x=2, y=0, z=-2)),
 2: (coord(x=2, y=2, z=-4), coord(x=0, y=-1, z=2)),
 3: (

{0: (coord(x=2, y=1, z=-5), coord(x=3, y=5, z=-4)),
 1: (coord(x=2, y=-8, z=2), coord(x=-2, y=-4, z=0)),
 2: (coord(x=2, y=-6, z=-1), coord(x=0, y=-3, z=0)),
 3: (coord(x=2, y=0, z=6), coord(x=-1, y=2, z=4))}
{0: (coord(x=5, y=3, z=-6), coord(x=3, y=2, z=-1)),
 1: (coord(x=0, y=-9, z=1), coord(x=-2, y=-1, z=-1)),
 2: (coord(x=2, y=-8, z=0), coord(x=0, y=-2, z=1)),
 3: (coord(x=1, y=1, z=7), coord(x=-1, y=1, z=1))}
{0: (coord(x=5, y=2, z=-4), coord(x=0, y=-1, z=2)),
 1: (coord(x=1, y=-7, z=-1), coord(x=1, y=2, z=-2)),
 2: (coord(x=1, y=-9, z=2), coord(x=-1, y=-1, z=2)),
 3: (coord(x=1, y=1, z=5), coord(x=0, y=0, z=-2))}
{0: (coord(x=2, y=-2, z=1), coord(x=-3, y=-4, z=5)),
 1: (coord(x=3, y=-4, z=-2), coord(x=2, y=3, z=-1)),
 2: (coord(x=1, y=-7, z=3), coord(x=0, y=2, z=1)),
 3: (coord(x=2, y=0, z=0), coord(x=1, y=-1, z=-5))}
{0: (coord(x=-1, y=-7, z=5), coord(x=-3, y=-5, z=4)),
 1: (coord(x=2, y=0, z=0), coord(x=-1, y=4, z=2)),
 2: (coord(x=4, y=-2, z=1), coord(x=3, y=5, z=-2)),
 3: (co

 1: (coord(x=2, y=0, z=0), coord(x=-2, y=4, z=4)),
 2: (coord(x=2, y=-2, z=1), coord(x=0, y=5, z=-4)),
 3: (coord(x=2, y=-4, z=-2), coord(x=-1, y=-4, z=-2))}
{0: (coord(x=5, y=-9, z=2), coord(x=3, y=-2, z=-1)),
 1: (coord(x=0, y=1, z=5), coord(x=-2, y=1, z=5)),
 2: (coord(x=2, y=2, z=-4), coord(x=0, y=4, z=-5)),
 3: (coord(x=1, y=-7, z=-1), coord(x=-1, y=-3, z=1))}
{0: (coord(x=5, y=-8, z=0), coord(x=0, y=1, z=-2)),
 1: (coord(x=1, y=1, z=7), coord(x=1, y=0, z=2)),
 2: (coord(x=1, y=3, z=-6), coord(x=-1, y=1, z=-2)),
 3: (coord(x=1, y=-9, z=1), coord(x=0, y=-2, z=2))}
{0: (coord(x=2, y=-6, z=-1), coord(x=-3, y=2, z=-1)),
 1: (coord(x=3, y=0, z=6), coord(x=2, y=-1, z=-1)),
 2: (coord(x=1, y=1, z=-5), coord(x=0, y=-2, z=1)),
 3: (coord(x=2, y=-8, z=2), coord(x=1, y=1, z=1))}
{0: (coord(x=-1, y=-3, z=-1), coord(x=-3, y=3, z=0)),
 1: (coord(x=2, y=-2, z=2), coord(x=-1, y=-2, z=-4)),
 2: (coord(x=4, y=-4, z=-1), coord(x=3, y=-5, z=4)),
 3: (coord(x=3, y=-4, z=2), coord(x=1, y=4, z=0))}
{0: 

{0: (coord(x=5, y=0, z=1), coord(x=3, y=0, z=3)),
 1: (coord(x=2, y=-10, z=-4), coord(x=1, y=0, z=1)),
 2: (coord(x=0, y=-8, z=5), coord(x=-3, y=0, z=-1)),
 3: (coord(x=1, y=5, z=0), coord(x=-1, y=0, z=-3))}
{0: (coord(x=5, y=-1, z=3), coord(x=0, y=-1, z=2)),
 1: (coord(x=2, y=-7, z=0), coord(x=0, y=3, z=4)),
 2: (coord(x=0, y=-7, z=1), coord(x=0, y=1, z=-4)),
 3: (coord(x=1, y=2, z=-2), coord(x=0, y=-3, z=-2))}
{0: (coord(x=2, y=-3, z=2), coord(x=-3, y=-2, z=-1)),
 1: (coord(x=1, y=-2, z=5), coord(x=-1, y=5, z=5)),
 2: (coord(x=3, y=-4, z=-4), coord(x=3, y=3, z=-5)),
 3: (coord(x=2, y=-4, z=-1), coord(x=1, y=-6, z=1))}
{0: (coord(x=-1, y=-6, z=0), coord(x=-3, y=-3, z=-2)),
 1: (coord(x=3, y=0, z=7), coord(x=2, y=2, z=2)),
 2: (coord(x=3, y=1, z=-6), coord(x=0, y=5, z=-2)),
 3: (coord(x=3, y=-8, z=1), coord(x=1, y=-4, z=2))}
{0: (coord(x=-1, y=-8, z=-1), coord(x=0, y=-2, z=-1)),
 1: (coord(x=4, y=1, z=6), coord(x=1, y=1, z=-1)),
 2: (coord(x=2, y=3, z=-5), coord(x=-1, y=2, z=1)),
 3: (

{0: (coord(x=-1, y=-8, z=6), coord(x=-3, y=-1, z=2)),
 1: (coord(x=2, y=5, z=3), coord(x=-1, y=3, z=-2)),
 2: (coord(x=4, y=0, z=-2), coord(x=3, y=1, z=2)),
 3: (coord(x=3, y=-10, z=-5), coord(x=1, y=-3, z=-2))}
{0: (coord(x=-1, y=-8, z=5), coord(x=0, y=0, z=-1)),
 1: (coord(x=2, y=5, z=0), coord(x=0, y=0, z=-3)),
 2: (coord(x=4, y=0, z=1), coord(x=0, y=0, z=3)),
 3: (coord(x=3, y=-10, z=-4), coord(x=0, y=0, z=1))}
{0: (coord(x=2, y=-7, z=1), coord(x=3, y=1, z=-4)),
 1: (coord(x=3, y=2, z=-2), coord(x=1, y=-3, z=-2)),
 2: (coord(x=1, y=-1, z=3), coord(x=-3, y=-1, z=2)),
 3: (coord(x=2, y=-7, z=0), coord(x=-1, y=3, z=4))}
{0: (coord(x=5, y=-4, z=-4), coord(x=3, y=3, z=-5)),
 1: (coord(x=1, y=-4, z=-1), coord(x=-2, y=-6, z=1)),
 2: (coord(x=1, y=-3, z=2), coord(x=0, y=-2, z=-1)),
 3: (coord(x=1, y=-2, z=5), coord(x=-1, y=5, z=5))}
{0: (coord(x=5, y=1, z=-6), coord(x=0, y=5, z=-2)),
 1: (coord(x=0, y=-8, z=1), coord(x=-1, y=-4, z=2)),
 2: (coord(x=2, y=-6, z=0), coord(x=1, y=-3, z=-2)),
 

{0: (coord(x=2, y=2, z=-4), coord(x=3, y=-1, z=2)),
 1: (coord(x=3, y=-7, z=-1), coord(x=1, y=2, z=-2)),
 2: (coord(x=1, y=-9, z=2), coord(x=-3, y=-1, z=2)),
 3: (coord(x=2, y=1, z=5), coord(x=-1, y=0, z=-2))}
{0: (coord(x=5, y=-2, z=1), coord(x=3, y=-4, z=5)),
 1: (coord(x=1, y=-4, z=-2), coord(x=-2, y=3, z=-1)),
 2: (coord(x=1, y=-7, z=3), coord(x=0, y=2, z=1)),
 3: (coord(x=1, y=0, z=0), coord(x=-1, y=-1, z=-5))}
{0: (coord(x=5, y=-7, z=5), coord(x=0, y=-5, z=4)),
 1: (coord(x=0, y=0, z=0), coord(x=-1, y=4, z=2)),
 2: (coord(x=2, y=-2, z=1), coord(x=1, y=5, z=-2)),
 3: (coord(x=1, y=-4, z=-4), coord(x=0, y=-4, z=-4))}
{0: (coord(x=2, y=-9, z=6), coord(x=-3, y=-2, z=1)),
 1: (coord(x=2, y=1, z=3), coord(x=2, y=1, z=3)),
 2: (coord(x=2, y=2, z=-2), coord(x=0, y=4, z=-3)),
 3: (coord(x=2, y=-7, z=-5), coord(x=1, y=-3, z=-1))}
{0: (coord(x=-1, y=-8, z=4), coord(x=-3, y=1, z=-2)),
 1: (coord(x=4, y=1, z=5), coord(x=2, y=0, z=2)),
 2: (coord(x=2, y=3, z=-4), coord(x=0, y=1, z=-2)),
 3: (c

 1: (coord(x=2, y=-4, z=0), coord(x=1, y=3, z=-2)),
 2: (coord(x=0, y=-7, z=1), coord(x=-3, y=2, z=2)),
 3: (coord(x=1, y=0, z=-4), coord(x=-1, y=-1, z=-6))}
{0: (coord(x=5, y=-7, z=8), coord(x=0, y=-5, z=3)),
 1: (coord(x=2, y=0, z=-1), coord(x=0, y=4, z=-1)),
 2: (coord(x=0, y=-2, z=2), coord(x=0, y=5, z=1)),
 3: (coord(x=1, y=-4, z=-7), coord(x=0, y=-4, z=-3))}
{0: (coord(x=2, y=-9, z=8), coord(x=-3, y=-2, z=0)),
 1: (coord(x=1, y=1, z=-1), coord(x=-1, y=1, z=0)),
 2: (coord(x=3, y=2, z=2), coord(x=3, y=4, z=0)),
 3: (coord(x=2, y=-7, z=-7), coord(x=1, y=-3, z=0))}
{0: (coord(x=-1, y=-8, z=5), coord(x=-3, y=1, z=-3)),
 1: (coord(x=3, y=1, z=0), coord(x=2, y=0, z=1)),
 2: (coord(x=3, y=3, z=1), coord(x=0, y=1, z=-1)),
 3: (coord(x=3, y=-9, z=-4), coord(x=1, y=-2, z=3))}
{0: (coord(x=-1, y=-6, z=-1), coord(x=0, y=2, z=-6)),
 1: (coord(x=4, y=0, z=2), coord(x=1, y=-1, z=2)),
 2: (coord(x=2, y=1, z=-1), coord(x=-1, y=-2, z=-2)),
 3: (coord(x=3, y=-8, z=2), coord(x=0, y=1, z=6))}
{0: (co

In [16]:
import re
from copy import deepcopy
from pprint import pprint
from itertools import repeat, permutations, product
from collections import namedtuple, defaultdict

coord = namedtuple('coord', ['x', 'y', 'z'])
print(inp)
moons = dict(enumerate(map(lambda x: dict(zip(['x', 'y', 'z'], zip(map(int, x.split(',')), [0,0,0]))), re.sub(r'[<>=xyz ]', '', inp).split('\n'))))
# pprint(moons)

pairs = set()
for p in product([0, 1, 2, 3], repeat=2):
    if (p not in pairs) and ((p[1], p[0]) not in pairs):
        pairs.add(p)
# pprint(pairs)

@memoize
def update_gravity(a, b):
    if a[0] > b[0]:
        a = (a[0], a[1] - 1)
        b = (b[0], b[1] + 1)
    elif a[0] < b[0]:
        a = (a[0], a[1] + 1)
        b = (b[0], b[1] - 1)
    return a, b

@memoize
def update_velocity(a):
    return (a[0] + a[1], a[1])


# for i in range(10):
#     for dim in ['x', 'y', 'z']:
#         for a, b in pairs:
#             moons[a][dim], moons[b][dim] = update_gravity(moons[a][dim], moons[b][dim])
#         for m in moons:
#    moons[m][dim] = update_velocity(moons[m][dim])
mem = set()
i = 0

moons = {
    'x': {k: v['x'] for k, v in moons.items()},
    'y': {k: v['y'] for k, v in moons.items()},
    'z': {k: v['z'] for k, v in moons.items()},
}
pprint(moons)
dim = 'z'
mem.add(tuple(moons[dim].values()))

while True:
    for a, b in pairs:
        moons[dim][a], moons[dim][b] = update_gravity(moons[dim][a], moons[dim][b])
    for m in moons[dim]:
        moons[dim][m] = update_velocity(moons[dim][m])
    i += 1
    hashed = tuple(moons[dim].values())
    if hashed in mem:
#         print(i, [k for k, v in mem.items() if v == str(moons[dim])][0], moons[dim])
        break
    else:
        mem.add(hashed)
    if i % 10000 == 0:
        print(i, len(mem))

print(i)

<x=16, y=-8, z=13>
<x=4, y=10, z=10>
<x=17, y=-5, z=6>
<x=13, y=-3, z=0>
{'x': {0: (16, 0), 1: (4, 0), 2: (17, 0), 3: (13, 0)},
 'y': {0: (-8, 0), 1: (10, 0), 2: (-5, 0), 3: (-3, 0)},
 'z': {0: (13, 0), 1: (10, 0), 2: (6, 0), 3: (0, 0)}}
10000 10001
20000 20001
30000 30001
40000 40001
50000 50001
60000 60001
70000 70001
80000 80001
90000 90001
100000 100001
102356


In [17]:
import math

def lcm(a, b):
    return abs(a * b) // math.gcd(a, b)

lcm(lcm(113028, 231614), 102356)

334945516288044

In [130]:
18 * 28 * 44
import math
# gcd(28, 44)


def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)
lcm(18, 28)
lcm(252, 44)

2772